In [1]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from scipy.fftpack import fft2, ifft2, fftshift
from math import log, exp

def extract_frames(video_path, max_frames=50):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while cap.isOpened() and count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized_frame = cv2.resize(gray_frame, (128, 128))
        frames.append(resized_frame)
        count += 1
    cap.release()
    return frames

def construct_log_gabor_filter(shape, center_freq, bandwidth):
    rows, cols = shape
    u = np.arange(-cols//2, cols//2)
    v = np.arange(-rows//2, rows//2)
    U, V = np.meshgrid(u, v)
    radius = np.sqrt(U**2 + V**2)
    radius[rows//2, cols//2] = 1
    log_radius = np.log(radius / center_freq)
    log_gabor = np.exp(-(log_radius**2) / (2 * (np.log(bandwidth))**2))
    log_gabor[radius == 0] = 0
    return fftshift(log_gabor)

def apply_log_gabor(image):
    image_fft = fft2(image)
    log_gabor_filter = construct_log_gabor_filter(image.shape, center_freq=10, bandwidth=1.5)
    filtered_fft = image_fft * log_gabor_filter
    filtered_image = np.abs(ifft2(filtered_fft))
    mean_val = np.mean(filtered_image)
    std_val = np.std(filtered_image)
    return np.array([mean_val, std_val])

def load_dataset_log_gabor(video_paths, labels, max_frames=50):
    features = []
    new_labels = []
    for i, video_path in enumerate(video_paths):
        if not os.path.exists(video_path):
            continue
        frames = extract_frames(video_path, max_frames)
        if len(frames) == 0:
            continue
        for frame in frames:
            feat = apply_log_gabor(frame)
            features.append(feat)
            new_labels.append(labels[i])
    return np.array(features), np.array(new_labels)

def train_random_forest(features, labels):
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, stratify=labels, random_state=42)
    clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy with Log-Gabor Features: {accuracy:.4f}")
    return clf

def prepare_ffpp_dataset(base_path):
    video_paths = []
    labels = []
    for label, folder in enumerate(['real', 'fake']):
        folder_path = os.path.join(base_path, folder)
        if not os.path.exists(folder_path):
            continue
        for video_file in os.listdir(folder_path):
            if video_file.endswith('.mp4'):
                video_paths.append(os.path.join(folder_path, video_file))
                labels.append(label)
    return video_paths, labels

base_path = "C:\\Users\\Pavan\\Downloads\\FF++"
video_paths, labels = prepare_ffpp_dataset(base_path)

try:
    features, labels = load_dataset_log_gabor(video_paths, labels)
    if len(features) > 0:
        model = train_random_forest(features, labels)
    else:
        print("No features extracted.")
except Exception as e:
    print(f"Error: {e}")


Accuracy with Log-Gabor Features: 0.7244
